# Cómo usar un cuaderno de Amazon SageMaker Studio

Este cuaderno lo guiará para que inicialice y configure el entorno. Muestra cómo cargar y descargar los datos desde y hacia un bucket de Amazon Simple Storage Service (Amazon S3) usando los métodos auxiliares de Amazon S3 del SDK para Python de SageMaker.

Para comenzar, inicialice el entorno. Para ello, importe las bibliotecas necesarias y obtenga el bucket de Amazon S3 predeterminado que utiliza SageMaker Studio.

In [ ]:
# Install dependencies
import pandas as pd
import boto3


# SageMaker dependencies
import sagemaker
from sagemaker import get_execution_role, session
from sagemaker.s3 import S3Downloader, S3Uploader

region= boto3.Session().region_name

# This object represents the AWS Identity and Access Management (IAM) role that you are assigned
role = sagemaker.get_execution_role()
print("Role: ", role)

sm_session = session.Session(boto3.Session())
sm = boto3.Session().client("sagemaker")
sm_runtime = boto3.Session().client("sagemaker-runtime")

Descargue el conjunto de datos del bucket de Amazon S3 usando el método S3Downloader del SDK para Python de SageMaker. Consulte [Método S3Downloader del SDK para Python de SageMaker](https://sagemaker.readthedocs.io/en/stable/api/utility/s3.html#sagemaker.s3.S3Downloader) para obtener más información sobre este método de descarga.

In [ ]:
bucket = ''
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        bucket = buckets.name
print(bucket)
prefix = 'scripts/data'

S3Downloader.download(s3_uri=f"s3://{bucket}/{prefix}/iris.csv", local_path= 'data/')

El siguiente código verifica el conjunto de datos y lo muestra en una cuadrícula.

In [ ]:
import pandas as pd
import numpy as np

shape=pd.read_csv("data/iris.csv", header=None)
shape.sample(3)

Cree particiones del conjunto de datos para obtener divisiones de entrenamiento y pruebas. Cárguelo en un bucket de Amazon S3 usando el método S3Uploader del SDK para Python de SageMaker. Consulte [Método S3Uploader del SDK para Python de SageMaker](https://sagemaker.readthedocs.io/en/stable/api/utility/s3.html#sagemaker.s3.S3Uploader) para obtener más información sobre este método de carga.

In [ ]:
train_data = shape.sample(frac=0.8,random_state=200)
test_data = shape.drop(train_data.index)

In [ ]:
train_file = 'train_data.csv';
train_data.to_csv(train_file, index=False, header=True)
test_file = 'test_data.csv';
test_data.to_csv(test_file, index=False, header=True)

# Return the URLs of the uploaded file, so they can be reviewed or used elsewhere
s3url = S3Uploader.upload(train_file, 's3://{}/{}'.format(bucket, prefix + "/train", "train"))
print(s3url)
s3url = S3Uploader.upload(test_file, 's3://{}/{}'.format(bucket, prefix + "/test", "test"))
print(s3url)

### Conclusión

¡Felicitaciones! Inicializó correctamente su entorno y cargó y descargó archivos de un bucket de Amazon S3. 

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, haga lo siguiente:

- Cierre el archivo del cuaderno
- Vuelva a las instrucciones del laboratorio